In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import sys
import pathlib

### Append directory to notebook

In [2]:
parent_dir = pathlib.Path(os.getcwd()).parent
sys.path.append(str(parent_dir))

In [4]:
from utils.util import load_model
from recommend.dataset import Dataset
from utils import variables

### Load model

In [7]:
als = load_model("../models/als.pkl")
cb = load_model("../models/cb.pkl")

In [9]:
def load_data(path):
    df = pd.read_csv(path, index_col=0)
    return Dataset(df, user="UserID", item="MovieID", rating="Rating")

In [10]:
# Load rating data to Dataset
ds = load_data("../data/rating.csv")

In [11]:
%%time
als.recommend(0, ds.get_csr()[0], N=20)

CPU times: total: 46.9 ms
Wall time: 52 ms


(array([ 390,  105,  167,  609,  104,  596,  207,  280,  632,  127,  273,
         124,  381,  128,  612,  588,   92, 1046,   85,  682]),
 array([1.088964  , 0.8012306 , 0.76091135, 0.7478144 , 0.74544036,
        0.742708  , 0.7148326 , 0.69618624, 0.68975836, 0.6547731 ,
        0.64351046, 0.64081377, 0.64046144, 0.6374482 , 0.63640064,
        0.61980534, 0.6041487 , 0.58303964, 0.5774966 , 0.5744972 ],
       dtype=float32))

In [12]:
def get_user_history(user_id):
    if user_id not in ds.user_list:
        raise ValueError(f"User id {user_id} not in dataset")
    return ds.get_user_history(user_id)

In [13]:
from sqlalchemy import create_engine

In [19]:
engine = create_engine("sqlite:///../data/movielens")

In [23]:
def read_data(query, engine):
    return pd.read_sql_query(query, engine)
def get_movie_information(iid, table=variables.TABLE, sql_engine=engine):
    query = f"""SELECT * FROM {table} WHERE MovieID = {iid}"""
    return read_data(query, engine)

In [24]:
def get_metadata(*items):
    res = []
    for item in items[0]:
        res.append(get_movie_information(item))
    return pd.concat(res, ignore_index=True)

In [25]:
get_metadata(als.recommend(0, ds.get_csr()[0], N=20)[0])

,index,MovieID,Title,Genres,release_year
0,386,390,Faster Pussycat! Kill! Kill!,"Action,Comedy,Drama",1965
1,103,105,"Bridges of Madison County, The","Drama,Romance",1995
2,165,167,Feast of July,Drama,1995
3,605,609,Homeward Bound II: Lost in San Francisco,"Adventure,Children's",1996
4,102,104,Happy Gilmore,Comedy,1996
5,592,596,Pinocchio,"Animation,Children's",1940
6,205,207,"Walk in the Clouds, A","Drama,Romance",1995
7,277,280,Murder in the First,"Drama,Thriller",1995
8,627,632,Land and Freedom (Tierra y libertad),War,1995
9,125,127,"Silence of the Palace, The (Saimt el Qusur)",Drama,1994


In [29]:
def get_res_result(user_id, topk=10):
    if user_id not in ds.user_list:
        raise ValueError(f"User id {user_id} not in dataset")
    idx = np.where(ds.user_list == user_id)[0][0]
    item_id, score = als.recommend(idx, ds.get_csr()[idx], N=topk)
    df = get_metadata(item_id)
    df['score'] = score
    return df

In [30]:
# check assert
get_res_result(0)

ValueError: User id 0 not in dataset

In [31]:
get_res_result(2)

,index,MovieID,Title,Genres,release_year,score
0,261,264,"Enfer, L'",Drama,1994,0.995726
1,22,23,Assassins,Thriller,1995,0.981739
2,193,195,Something to Talk About,"Comedy,Drama,Romance",1995,0.967729
3,187,189,Reckless,Comedy,1995,0.956880
4,40,41,Richard III,"Drama,War",1995,0.917832
5,586,590,Dances with Wolves,"Adventure,Drama,Western",1990,0.852498
6,501,505,North,Comedy,1994,0.847792
7,755,765,Jack,"Comedy,Drama",1996,0.841336
8,277,280,Murder in the First,"Drama,Thriller",1995,0.808011
9,428,432,City Slickers II: The Legend of Curly's Gold,"Comedy,Western",1994,0.796700


In [32]:
def parse_cb_result(model, iid, topk=10):
    res_id, res_score = model.recommend(iid, topk)
    df = get_metadata(res_id)
    df['score'] = res_score
    return df

In [34]:
parse_cb_result(cb, 2)

,index,MovieID,Title,Genres,release_year,score
0,1,2,Jumanji,"Adventure,Children's,Fantasy",1995,0.666667
1,59,60,"Indian in the Cupboard, The","Adventure,Children's,Fantasy",1995,0.666667
2,55,56,Kids of the Round Table,"Adventure,Children's,Fantasy",1995,0.666667
3,7,8,Tom and Huck,"Adventure,Children's",1995,0.637624
4,235,238,Far From Home: The Adventures of Yellow Dog,"Adventure,Children's",1995,0.637624
5,744,754,Gold Diggers: The Secret of Bear Mountain,"Adventure,Children's",1995,0.637624
6,144,146,"Amazing Panda Adventure, The","Adventure,Children's",1995,0.637624
7,156,158,Casper,"Adventure,Children's",1995,0.637624
8,255,258,"Kid in King Arthur's Court, A","Adventure,Children's,Comedy,Fantasy,Romance",1995,0.634845
9,0,1,Toy Story,"Animation,Children's,Comedy",1995,0.619164
